In [1]:
import importlib
import full_model

In [2]:
full_model.eng = full_model.start_matlab()


Starting MATLAB engine...
-------------------------
Elapsed: 250.59 sec


In [3]:
images_norm = full_model.loadmat("./saved/yaleB.mat")['X']
labels = full_model.loadmat("./saved/yaleB.mat")['Y'].reshape(-1)

In [4]:
from skopt.space import Real, Integer

space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(2, 3, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(2, 3, name='maxIter2'),]

In [5]:
from skopt.utils import use_named_args

@use_named_args(space)
def objective(**params):
    try:
        #return 1-full_model.run_model(images_norm, labels, seed, ep_prtr, ep, **params)[0]
        return params['lr_pretrain']*params['lr']*params['alpha1']*params['alpha2']
    except Exception as ex:
        if(type(ex) == KeyError):
            raise ex
        return 1

In [245]:
from skopt import gp_minimize, dummy_minimize
from skopt.plots import plot_convergence
import time

def optimize(function, iterations, epochs_pretrain, epochs, random_seed):
    start_time = time.time()
    
    # global b/c I couldn't find a better way to directly pass these to the objective function
    global ep_prtr, ep, seed
    ep_prtr = epochs_pretrain
    ep = epochs
    seed = random_seed
    
    # kwargs b/c dummy_minimize can not take n_jobs
    kwargs = {"n_calls":iterations, "random_state":random_seed, "verbose":True, "callback":callback_func}
    if(function != dummy_minimize):
        kwargs["n_jobs"] = -1
    res = function(objective, space, **kwargs)
    end_func(res)
    opt_stats(res, start_time)
    return res

In [18]:
def opt_stats(res, start_time=None):
    plot_convergence(res, yscale="log")
    print("\n------------------")
    for i in range(10, len(res.func_vals)+1, 10):
        print("{0:d}: {1:.4f}".format(i, min(res.func_vals[0:i])))
    print("Best score: {0:.4f}".format(res.fun))
    print("Best parameters: {0:}".format(res.x))
    if(start_time is not None):
        print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))

In [71]:
def asdf(result, prt=True):
    print(len(result.func_vals))
    if(prt):
        print(result.random_state.bytes(10))
    if(len(result.models) > 0):
        print(result.models[-1].predict(np.zeros((1, 6))))

In [268]:
def end_func(result):
    global random_current_, random_prev_
    if("random_prev_" in globals()):
        result.random_state = random_prev_
    else:
        raise Exception("Not enough iterations")
    del random_current_
    del random_prev_

In [267]:
random_current_

In [266]:
callback_func(res1)

False

In [284]:
from copy import copy

def callback_func(result):
    asdf(result)
    global random_current_, random_prev_
    if("random_current_" in globals()):
        random_prev_ = random_current_
    random_current_ = copy(result.random_state)
    return False

In [50]:
# set models to that of the previous optimization
        result.models[:0] = models[:-1]
        # the currently active model is stored internally in the optimization
        # we can access it here, but it can't be explicitly replaced
        # we need to keep it as the same object, modifying its parameters from the inside
        model_old = models[-1]
        model_new = result.models[-1]
        model_new.set_params(**model_old.get_params())
        # now, manually set all parameters that set_params() can't reach
        model_new.K_inv_ = model_old.K_inv_
        model_new.L_ = model_old.L_
        model_new.X_train_ = model_old.X_train_
        model_new.alpha_ = model_old.alpha_
        model_new.kernel_ = model_old.kernel_
        model_new.log_marginal_likelihood_value_ = model_old.log_marginal_likelihood_value_
        model_new.noise_ = model_old.noise_
        model_new.y_train_ = model_old.y_train_
        model_new.y_train_mean_ = model_old.y_train_mean_

IndentationError: unexpected indent (<ipython-input-50-d8e367b5da37>, line 2)

In [70]:
def helper(result):
    global models, state, set_vars
    if(set_vars):
        # same situation with the RandomState object
        # modifying its parameters like this makes the RandomState object behave identically,
        # while keeping it at the same location in memory
        # thus, pointers in any other parts of the code will still receive this new object
        result.random_state.set_state(state)
        
        # this only needs to be performed once, after the first iteration
        set_vars = False
        asdf(result, False)
    else:
        asdf(result, True)

In [274]:
from copy import copy

def reload(result, addtl_calls):
    start_time = time.time()
    args = copy(result.specs['args'])
    args['n_calls'] = addtl_calls
    args['n_random_starts'] = 0
    args['x0'] = result.x_iters
    args['y0'] = result.func_vals
    args['random_state'] = result.random_state
    result_new = gp_minimize(**args)
    end_func(result_new)
    opt_stats(result_new, start_time)
    return result_new

In [ ]:
res1 = optimize(gp_minimize, 12, 1, 1, 0)

In [ ]:
res12 = reload(res1, 4)

In [ ]:
res2 = optimize(gp_minimize, 16, 1, 1, 0)

In [21]:
from skopt import load
seed_ = 1
iters_ = 50
gp_loaded = load("optims/scenario0/gp_" + str(seed_) + "_" + str(iters_) + ".opt")
dummy_loaded = load("optims/scenario0/dummy_" + str(seed_) + "_" + str(iters_) + ".opt")
forest_loaded = load("optims/scenario0/forest_" + str(seed_) + "_" + str(iters_) + ".opt")
gbrt_loaded = load("optims/scenario0/gbrt_" + str(seed_) + "_" + str(iters_) + ".opt")

In [ ]:
opt_stats(gbrt_loaded)